In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import json, io, base64, os
import plotly.graph_objects as go
from IPython.display import display, HTML, clear_output
import plotly.express as px
import ipywidgets as widgets

# Initialize settings
import _settings as settings

# Global variables from settings
plotly_colors = settings.plotly_colors


In [2]:
class DataTransformation:
    def __init__(self):
        self.merged_df = None
        self.group_data = None
        self.output_area = None
        self.merged_uploader = None
        self.group_uploader = None
        self.reset_button = None
        self.callbacks = []  # List to store callbacks
        self.filtered_df = None
        self.filter_output = widgets.Output()
        self.status_area = widgets.Output()
                
    def _update_filter_options(self):
        """Update dropdown options based on loaded data"""
        if self.merged_df is None:
            print("No data loaded yet, can't update filter options")
            return
            
        # Save current selections if any
        current_proteins = self.protein_dropdown.value if hasattr(self, 'protein_dropdown') and self.protein_dropdown else ['All']
        current_functions = self.function_dropdown.value if hasattr(self, 'function_dropdown') and self.function_dropdown else ['All']
        
        # Get unique proteins if the column exists and count occurrences
        if 'Master Protein Accessions' in self.merged_df.columns:
            # Dictionary to store protein counts
            protein_counts = {}
            
            # Count occurrences of each protein
            for acc in self.merged_df['Master Protein Accessions'].dropna():
                for protein in [p.strip() for p in str(acc).split(';') if p.strip()]:
                    if protein in protein_counts:
                        protein_counts[protein] += 1
                    else:
                        protein_counts[protein] = 1
            
            # Sort proteins by count (descending)
            sorted_proteins = sorted(protein_counts.items(), key=lambda x: x[1], reverse=True)
            
            # Extract just the protein names in sorted order
            proteins = [protein for protein, count in sorted_proteins]
            
            self.protein_dropdown.options = ['All'] + proteins
            self.protein_dropdown.disabled = False
        else:
            print("No 'Master Protein Accessions' column found")
            self.protein_dropdown.options = ['All']
            self.protein_dropdown.disabled = True
        
        # Check for function column with various capitalizations
        function_col = None
        for col in ['function', 'Function', 'FUNCTION']:
            if col in self.merged_df.columns:
                function_col = col
                break
        
        if function_col:
            # Dictionary to store function counts
            function_counts = {}
            
            # Count occurrences of each function
            for func_str in self.merged_df[function_col].dropna():
                for function in [f.strip() for f in str(func_str).split(';') if f.strip()]:
                    if function in function_counts:
                        function_counts[function] += 1
                    else:
                        function_counts[function] = 1
            
            # Sort functions by count (descending)
            sorted_functions = sorted(function_counts.items(), key=lambda x: x[1], reverse=True)
            
            # Extract just the function names in sorted order
            functions = [function for function, count in sorted_functions]
            
            
            if len(functions) > 0:
                self.function_dropdown.options = ['All'] + functions
                self.function_dropdown.disabled = False
            else:
                print("Function column exists but has no data")
                self.function_dropdown.options = ['All']
                self.function_dropdown.disabled = True
        else:
            print("No function column found")
            self.function_dropdown.options = ['All']
            self.function_dropdown.disabled = True
        
        # Restore selections if still valid or default to 'All'
        if 'All' in current_proteins or any(p in proteins for p in current_proteins if 'Master Protein Accessions' in self.merged_df.columns):
            self.protein_dropdown.value = current_proteins
        else:
            self.protein_dropdown.value = ['All']
            
        if function_col and ('All' in current_functions or any(f in functions for f in current_functions)):
            self.function_dropdown.value = current_functions
        else:
            self.function_dropdown.value = ['All']

    def _apply_filters(self):
        """Apply selected filters to the data"""
        if self.merged_df is None:
            return
                
        self.filtered_df = self.merged_df.copy()
        
        # Apply protein filter if column exists
        if (self.protein_dropdown and 
            'All' not in self.protein_dropdown.value and 
            'Master Protein Accessions' in self.filtered_df.columns):
            
            selected_proteins = self.protein_dropdown.value
            mask = self.filtered_df['Master Protein Accessions'].fillna('').apply(
                lambda x: any(protein in [p.strip() for p in str(x).split(';')] for protein in selected_proteins)
            )
            self.filtered_df = self.filtered_df[mask]
        
        # Apply function filter - check for both 'Function' and 'function' columns
        if self.function_dropdown and 'All' not in self.function_dropdown.value:
            function_col = None
            for col_name in ['function', 'Function', 'FUNCTION']:
                if col_name in self.filtered_df.columns:
                    function_col = col_name
                    break
                    
            if function_col:
                selected_functions = self.function_dropdown.value
                
                # Create a mask for rows that contain any of the selected functions
                mask = self.filtered_df[function_col].fillna('').apply(
                    lambda x: any(func in [f.strip() for f in str(x).split(';')] for func in selected_functions)
                )
                self.filtered_df = self.filtered_df[mask]

    def _on_filter_change(self, change):
        """Handle changes in filter selections"""
        with self.filter_output:
            self.filter_output.clear_output()
            
            self._apply_filters()
            
            # Show filter status
            total_rows = len(self.filtered_df) if self.filtered_df is not None else 0
            if total_rows > 0 and self.plot_func_or_pro.value != 'None':
                display(HTML(
                    f'<div style="color: green; margin-top: 10px;">'
                    f'Filtered data: {total_rows} rows</div>'
                ))

            self._trigger_callbacks()

    def setup_data_loading_ui(self):
        """Initialize and display the data loading UI"""
        self.merged_uploader = widgets.FileUpload(
            accept='.csv,.txt,.tsv,.xlsx',
            multiple=False,
            description='Upload Merged Data File',
            layout=widgets.Layout(width='300px'),
            style={'description_width': 'initial'}
        )
        
        self.group_uploader = widgets.FileUpload(
            accept='.json',
            multiple=False,
            description='Upload Group Definition',
            layout=widgets.Layout(width='300px'),
            style={'description_width': 'initial'}
        )

        self.output_area = widgets.Output()

        merged_box = widgets.HBox([
            self.merged_uploader,
            self.create_download_link("example_merged_dataframe.csv", "Example")
        ], layout=widgets.Layout(align_items='center'))

        group_box = widgets.HBox([
            self.group_uploader,
            self.create_download_link("example_group_definition.json", "Example")
        ], layout=widgets.Layout(align_items='center'))
        
        self.upload_widgets = widgets.VBox([
            widgets.HTML("<h4>Upload Data Files:</h4>"),
            merged_box,
            widgets.HTML("<h4>Upload Group Definition:</h4>"),
            group_box,
            self.output_area
        ])
        
        # Replace Dropdown with SelectMultiple for proteins
        self.protein_dropdown = widgets.SelectMultiple(
            options=['All'],
            value=['All'],
            #description='Proteins: ',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', height='100px')  # Added height for multiple selections
        )
        
        # Replace Dropdown with SelectMultiple for functions
        self.function_dropdown = widgets.SelectMultiple(
            options=['All'],
            value=['All'],
            #description='Functions:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', height='100px')  # Added height for multiple selections
        )
        
        # Create filter container
        self.filter_container = widgets.VBox([
            self.function_dropdown,
            self.protein_dropdown,
            ], layout=widgets.Layout(justify_content='flex-start')
        )
        
        self.status_area = widgets.Output()
              
        self.merged_uploader.observe(self._on_merged_upload_change, names='value')
        self.group_uploader.observe(self._on_group_upload_change, names='value')

        # Add observers for the dropdowns
        self.protein_dropdown.observe(self._on_filter_change, names='value')
        self.function_dropdown.observe(self._on_filter_change, names='value')
        
        display(self.upload_widgets,
                self.status_area)
    
    def _on_merged_upload_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            with self.output_area:
                self.output_area.clear_output()
                uploaded_files = change.get('new', ())
                if uploaded_files:
                    file_data = uploaded_files[0]
                    self.merged_df = self._load_merged_data(file_data)
                    
                    if self.merged_df is not None:   
                        # Check specifically for 'function' column
                        function_col = None
                        for col in ['function', 'Function', 'FUNCTION']:
                            if col in self.merged_df.columns:
                                function_col = col
                                break

                        # Always initialize filtered_df with the full data
                        self.filtered_df = self.merged_df.copy()
                        
                        display(HTML(
                            f'<b style="color:green;">Merged data imported: '
                            f'{self.merged_df.shape[0]} rows, {self.merged_df.shape[1]} columns</b>'
                        ))
                        
                        # Update filter options after loading data with sorted lists
                        self._update_filter_options()
                        
                        # Make sure the dropdowns appear right away
                        if hasattr(self, 'callbacks') and self.callbacks:
                            self._trigger_callbacks()
 
    def add_callback(self, callback):
        """Add a callback to be triggered when data changes"""
        self.callbacks.append(callback)
        
    def _trigger_callbacks(self):
        """Trigger all registered callbacks"""
        for callback in self.callbacks:
            callback()
            
    def create_download_link(self, file_path, label):
        """Create a download link for a file."""
        if os.path.exists(file_path):
            with open(file_path, 'rb') as f:
                content = f.read()
            b64_content = base64.b64encode(content).decode('utf-8')

            return widgets.HTML(f"""
                <a download="{os.path.basename(file_path)}" 
                   href="data:application/octet-stream;base64,{b64_content}" 
                   style="color: #0366d6; text-decoration: none; margin-left: 20px; font-size: 14px;">
                    {label}
                </a>
            """)
        else:
            return widgets.HTML(f"""
                <span style="color: red; margin-left: 20px; font-size: 14px;">
                    File "{file_path}" not found!
                </span>
            """)

    def _on_group_upload_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            with self.output_area:
                self.output_area.clear_output()
                uploaded_files = change.get('new', ())
                if uploaded_files:
                    file_data = uploaded_files[0]
                    try:
                        content = bytes(file_data.content).decode('utf-8')
                        group_data = json.loads(content)
                        self.group_data = self._process_group_data(group_data)
                        display(HTML(
                            f'<b style="color:green;">Group definition imported successfully with {len(self.group_data)} groups.</b><br>'
                        ))
                        self._trigger_callbacks()
                    except Exception as e:
                        display(HTML(f'<b style="color:red;">Error loading group definition: {str(e)}</b>'))

    def _load_merged_data(self, file_data):
        try:
            content = bytes(file_data.content)
            filename = file_data.name
            extension = filename.split('.')[-1].lower()
            
            file_stream = io.BytesIO(content)
            
            if extension == 'csv':
                df = pd.read_csv(file_stream)
            elif extension in ['txt', 'tsv']:
                df = pd.read_csv(file_stream, delimiter='\t')
            elif extension == 'xlsx':
                df = pd.read_excel(file_stream)
            else:
                raise ValueError("Unsupported file format")
            
            return df
            
        except Exception as e:
            display(HTML(f'<b style="color:red;">Error loading data: {str(e)}</b>'))
            return None

    def _process_group_data(self, json_data):
        """Process and validate the group data structure"""
        try:
            processed_data = {}
            for group_id, group_info in json_data.items():
                if 'grouping_variable' not in group_info:
                    raise ValueError(f"Group {group_id} missing grouping_variable")
                if 'abundance_columns' not in group_info:
                    raise ValueError(f"Group {group_id} missing abundance_columns")

                processed_data[group_id] = {
                    'grouping_variable': group_info['grouping_variable'],
                    'abundance_columns': group_info['abundance_columns']
                }

            return processed_data
        except Exception as e:
            raise ValueError(f"Error processing group data: {str(e)}")

In [3]:
class TotalPeptidePlotter:
    def __init__(self, data_transformer):
        self.data_transformer = data_transformer
        self.plot_output = widgets.Output()
        self.export_output = widgets.Output()
        self.info_output = widgets.Output()
        self.current_fig_abundance = None
        self.current_fig_count = None
        
        # Initialize widgets
        self.setup_widgets()
        
        # Register callback for data changes
        self.data_transformer.add_callback(self._update_group_options)
   
    def update_display(self, change=None):
        """Update the displayed dropdowns based on the user's selection."""
        with self.dropdown_output:
            self.dropdown_output.clear_output(wait=True)
            selected_value = self.plot_func_or_pro.value
            
            # Determine which dropdowns to display
            if selected_value == 'Selected Protein(s)':
                self.data_transformer.function_dropdown.layout.display = 'none'
                self.data_transformer.protein_dropdown.layout.display = 'block'
                
                # Make sure protein dropdown is initialized and populated
                if not hasattr(self.data_transformer, 'protein_dropdown') or self.data_transformer.protein_dropdown is None:
                    print("Protein dropdown not initialized")
                    return
                    
                grid = widgets.VBox([
                    widgets.HTML("<u>Proteins:</u>"),
                    self.data_transformer.protein_dropdown
                ])
                
                # Reset function dropdown to 'All'
                if hasattr(self.data_transformer, 'function_dropdown') and self.data_transformer.function_dropdown is not None:
                    self.data_transformer.function_dropdown.value = ['All']


            elif selected_value == 'Selected Function(s)':
                self.data_transformer.function_dropdown.layout.display = 'block'
                self.data_transformer.protein_dropdown.layout.display = 'none'
                
                # Make sure function dropdown is initialized and populated
                if not hasattr(self.data_transformer, 'function_dropdown') or self.data_transformer.function_dropdown is None:
                    print("Function dropdown not initialized")
                    return
                self.data_transformer.function_dropdown.layout.display = 'block'
                self.data_transformer.protein_dropdown.layout.display = 'block'  
                
                grid = widgets.VBox([
                    widgets.HTML("<u>Functions:</u>"),
                    self.data_transformer.function_dropdown
                ])

                # Reset protein dropdown to 'All'
                if hasattr(self.data_transformer, 'protein_dropdown') and self.data_transformer.protein_dropdown is not None:
                    self.data_transformer.protein_dropdown.value = ['All']

            elif selected_value == 'Both':
                self.data_transformer.function_dropdown.layout.display = 'block'
                self.data_transformer.protein_dropdown.layout.display = 'block'
                # Make sure both dropdowns are initialized
                if (not hasattr(self.data_transformer, 'protein_dropdown') or 
                    not hasattr(self.data_transformer, 'function_dropdown') or
                    self.data_transformer.protein_dropdown is None or 
                    self.data_transformer.function_dropdown is None):
                    print("One or both dropdowns not initialized")
                    return
                    
                grid = widgets.GridBox(
                    [
                        widgets.VBox([widgets.HTML("<u>Proteins:</u>"), self.data_transformer.protein_dropdown]),
                        widgets.VBox([widgets.HTML("<u>Functions:</u>"), self.data_transformer.function_dropdown])
                    ],
                    layout=widgets.Layout(
                        grid_template_columns="1fr 1fr",  # Two equal columns
                        gap="1px",                        # 1px spacing between elements
                        max_width="625px",                # Set maximum width to 620px
                        height="125px"                     # Auto height to adjust dynamically
                    )
                )               
            else:
                # Reset both dropdowns to 'All'
                if hasattr(self.data_transformer, 'protein_dropdown') and self.data_transformer.protein_dropdown is not None:
                    self.data_transformer.protein_dropdown.value = ['All']
                if hasattr(self.data_transformer, 'function_dropdown') and self.data_transformer.function_dropdown is not None:
                    self.data_transformer.function_dropdown.value = ['All']

                self.data_transformer.function_dropdown.layout.display = 'none'
                self.data_transformer.protein_dropdown.layout.display = 'none'
                grid = widgets.GridBox(
                    [
                        widgets.VBox([widgets.HTML(" "), self.data_transformer.protein_dropdown]),
                        widgets.VBox([widgets.HTML(" "), self.data_transformer.function_dropdown])
                    ],
                    layout=widgets.Layout(
                        grid_template_columns="1fr 1fr",  # Two equal columns
                        gap="1px",                        # 1px spacing between elements
                        max_width="625px",                # Set maximum width to 620px
                        height="10px"                     # Auto height to adjust dynamically
                    )
                )       
            if grid:
                display(grid)
 
    def setup_widgets(self):
        """Initialize UI widgets"""
        # Color pickers for bars
        self.abundance_color = widgets.ColorPicker(
            description='Absorbance Bar Color:',
            value='#0072C6',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.count_color = widgets.ColorPicker(
            description='Count Bar Color:',
            value='#0072C6',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Download HTML button
        self.download_html_button = widgets.Button(
            description='Download Interactive Plots',
            button_style='info',
            icon='file',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        # Create plot control widgets
        self.plot_button = widgets.Button(
            description='Generate/Update Plots',
            button_style='success',
            icon='refresh',
            layout=widgets.Layout(width='200px')
        )
        
        self.export_button = widgets.Button(
            description='Export Data',
            button_style='info',
            icon='download',
            layout=widgets.Layout(width='200px'),
            disabled=True
        )
        
        # Add group selection widget
        self.group_select = widgets.SelectMultiple(
            description='Groups:',
            options=[],
            layout=widgets.Layout(width='300px', height='100px'),
            style={'description_width': 'initial'}
        )
        
        # Plot type selection
        self.plot_type = widgets.Dropdown(
            options=['Log Scale', 'Linear Scale'],
            value='Log Scale',
            description='Y-axis Scale:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )

        # Add label customization
        self.xlabel_widget = widgets.Text(
            description='X Label:',
            placeholder='Enter custom label',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.ylabel_abundance_widget = widgets.Text(
            description='Absorbance Y Label:',
            placeholder='Enter custom label',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.ylabel_count_widget = widgets.Text(
            description='Count Y Label:',
            placeholder='Enter custom label',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        self.title_widget = widgets.Text(
            description='Plot Title:',
            placeholder='Enter custom title',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
       # Plot type selection
        self.plot_func_or_pro = widgets.Dropdown(
            options=['None', 'Selected Protein(s)', 'Selected Function(s)', 'Both'],
            value='None',
            description='Plot Filter:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Output widget to hold the dropdowns
        self.dropdown_output = widgets.Output()

        # Observe changes in the dropdown selection
        self.plot_func_or_pro.observe(self.update_display, names='value')
        # No need for update_widgets function as we're handling everything in update_display
        
        

        # Create layout with sections
        controls_box = widgets.VBox([
            widgets.HTML("<h4>Plot Controls:</h4>"),
            self.group_select,
            self.plot_func_or_pro,
            self.dropdown_output,
            self.data_transformer.filter_output,
            self.plot_type
        ])
        
        appearance_box = widgets.VBox([
            widgets.HTML("<h4>Appearance Settings:</h4>"),
            self.xlabel_widget,
            self.ylabel_abundance_widget,
            self.ylabel_count_widget,
            self.title_widget
        ])
        
        # Add color customization section
        color_box = widgets.VBox([
            self.abundance_color,
            self.count_color
        ])
        
        button_box = widgets.VBox([
            widgets.HTML("<h4>Actions:</h4>"),
            widgets.VBox([
                self.plot_button,
                self.export_button,
                self.download_html_button
            ])
        ])
        
        # Create main layout
        self.widget_box = widgets.VBox([
            controls_box,
            appearance_box,
            color_box,
            button_box,
            self.info_output,
            self.plot_output,
            self.export_output
        ])
        
        # Add button callbacks
        self.plot_button.on_click(self._on_plot_button_click)
        self.export_button.on_click(self._on_export_button_click)
        self.download_html_button.on_click(self._on_download_html_click)
        
        # Make sure we're not calling the plot function directly on plot_type change
        # This can cause issues with filters
        if hasattr(self, '_plot_type_observe_callback'):
            self.plot_type.unobserve(self._plot_type_observe_callback, names='value')
        self._plot_type_observe_callback = lambda change: self._refresh_plot_if_exists(change)
        self.plot_type.observe(self._plot_type_observe_callback, names='value')

        # Add observer for data changes
        self.data_transformer.merged_uploader.observe(self._update_group_options, names='value')
   
    def _refresh_plot_if_exists(self, change):
        """Only refresh plot if it already exists"""
        if hasattr(self, 'current_fig_abundance') and self.current_fig_abundance is not None:
            self._on_plot_button_click(None)
            

    def _update_group_options(self, change=None):
        """Update group selection options when data changes"""
        if self.data_transformer.merged_df is not None and self.data_transformer.group_data is not None:
            # Get grouping variables from the group data
            group_options = [
                info['grouping_variable'] 
                for info in self.data_transformer.group_data.values()
            ]
            
            # Update group selection options
            self.group_select.options = group_options
            # Select all groups by default
            self.group_select.value = group_options
                         
    def _process_total_peptide_data(self):
        """Process peptide data for visualization using group definitions"""
        if self.data_transformer.merged_df is None or self.data_transformer.group_data is None:
            return None
        else:
            df = self.data_transformer.filtered_df.copy()  # Create a copy to avoid modifying original
        
        results = {}
        
        # Process each group from the group_data dictionary
        for group_id, group_info in self.data_transformer.group_data.items():
            group_name = group_info['grouping_variable']
            abundance_columns = group_info['abundance_columns']
            
            # Calculate total abundance and SEM from the abundance columns
            valid_abundance_cols = [col for col in abundance_columns if col in df.columns]
            
            if not valid_abundance_cols:
                print(f"Warning: No valid abundance columns found for group {group_name}")
                continue
                    
            # Filter for non-zero, non-null values in any abundance column
            temp_df = df[['unique ID'] + valid_abundance_cols].copy()
            
            # Convert abundance columns to numeric, forcing non-numeric values to NaN
            for col in valid_abundance_cols:
                temp_df[col] = pd.to_numeric(temp_df[col], errors='coerce')
            
            temp_df = temp_df[
                temp_df[valid_abundance_cols].notna().any(axis=1) & 
                (temp_df[valid_abundance_cols] != 0).any(axis=1) &
                temp_df['unique ID'].notna()
            ]
            
            if temp_df.empty:
                continue
                    
            # Calculate peptide counts for each replicate
            replicate_counts = []
            for col in valid_abundance_cols:
                count = temp_df[temp_df[col].notna() & (temp_df[col] != 0)]['unique ID'].nunique()
                replicate_counts.append(count)
            
            # Calculate mean count and SEM across replicates
            if len(replicate_counts) > 1:
                count_sem = np.std(replicate_counts, ddof=1) / np.sqrt(len(replicate_counts))
            else:
                count_sem = 0
                
            # Calculate abundance statistics
            abundances = temp_df[valid_abundance_cols].values.astype(float)  # Ensure float type
            peptide_means = np.nanmean(abundances, axis=1)
            total_abundance = np.nansum(peptide_means)
            
            # Calculate SEM for abundance
            peptide_sems = np.nanstd(abundances, axis=1) / np.sqrt(abundances.shape[1])
            total_sem = np.sqrt(np.nansum(peptide_sems ** 2))

            #calc total count for group
            unique_peptides = np.mean(replicate_counts)
            
            all_unique_peptides = temp_df[
                (temp_df[valid_abundance_cols] > 0).any(axis=1)
                ]['unique ID'].nunique()
            
            results[group_name] = {
                'unique_peptides': all_unique_peptides,
                'total_Absorbance': total_abundance,
                'total_sem': total_sem,
                'abundance_sem': total_sem,
                'count_sem': count_sem,
                'replicate_data': {
                    'abundance_columns': valid_abundance_cols,
                    'replicate_counts': replicate_counts,
                    'replicate_abundances': [temp_df[col].replace(0, np.nan).sum() for col in valid_abundance_cols]
                }
            }
        
        return results

    def plot_total_peptides(self, data):
        """Generate interactive Plotly bar plots for total peptides.
        
        Args:
            data (dict): Dictionary containing peptide data with abundance and count information
            
        Returns:
            tuple: (abundance_figure, count_figure) Plotly figure objects
        """
        if not data:
            return None, None
        
        # Common styling configurations
        COMMON_LAYOUT = {
            'template': 'plotly_white',
            'height': 800,
            'width': 1000,
            'margin': dict(t=100, l=100, r=100),
            'showlegend': False,
            'font': {'color': 'black'},
        }
        
        AXIS_STYLE = {
            'showline': True,
            'linewidth': 1,
            'linecolor': 'black',
            'mirror': False,
            'gridcolor': 'lightgray',
            'showgrid': True,
            'zeroline': False,
        }
        
        def create_title(text):
            return {
                'text': text,
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {'size': 18, 'color': 'black'}
            }

        # Prepare data
        groups = list(data.keys())
        plot_data = {
            'abundances': [data[group]['total_Absorbance'] for group in groups],
            'abundance_sems': [data[group]['abundance_sem'] for group in groups],
            'counts': [data[group]['unique_peptides'] for group in groups],
            'count_sems': [data[group]['count_sem'] for group in groups]
        }
                
        # Create dynamic titles for Plotly with proper line breaks
        base_title = self.title_widget.value or 'Total Peptide'
        protein_title = ""
        function_title = ""
        function_filter_prefix = ""

        # Add protein filter info if applicable
        if (
            hasattr(self.data_transformer, 'protein_dropdown')
            and self.data_transformer.protein_dropdown
            and 'All' not in self.data_transformer.protein_dropdown.value
        ):
            protein_names = ", ".join(self.data_transformer.protein_dropdown.value)
            protein_title = f"Protein(s): {protein_names}"
            function_filter_prefix = "Filtered By:<br>"

        # Add function filter info if applicable
        if (
            hasattr(self.data_transformer, 'function_dropdown')
            and self.data_transformer.function_dropdown
            and 'All' not in self.data_transformer.function_dropdown.value
        ):
            function_names = ", ".join(self.data_transformer.function_dropdown.value)
            function_title = f"Function(s): {function_names}"
            function_filter_prefix = "- Filtered By:"

        # Combine filter titles neatly with <br> for Plotly
        filters_combined = "<br>".join(filter(None, [function_filter_prefix, protein_title, function_title]))

        # Create final dynamic titles for Plotly
        abundance_title = self.title_widget.value or f"{base_title} Absorbance Distribution {filters_combined}"
        count_title = self.title_widget.value or  f"{base_title} Count Distribution {filters_combined}"

        # Create abundance figure
        fig_abundance = go.Figure()
        
        # Add abundance bars
        fig_abundance.add_trace(go.Bar(
            x=groups,
            y=plot_data['abundances'],
            name='Total Absorbance',
            marker=dict(
                color=self.abundance_color.value,
                line=dict(color='black', width=1)
            ),
            error_y=dict(
                type='data',
                array=plot_data['abundance_sems'],
                visible=True,
                thickness=1.5,
                width=4,
                color='#000000'
            ),
            hovertemplate=(
                "Group: %{x}<br>"
                "Total Abundance: %{y:.2e}<br>"
                "SEM: %{error_y.array:.2e}<br>"
                "<extra></extra>"
            )
        ))
        
        # Add abundance labels
        fig_abundance.add_trace(go.Scatter(
            x=groups,
            y=[a + s for a, s in zip(plot_data['abundances'], plot_data['abundance_sems'])],
            mode='text',
            text=[f"{a:.2e}" for a in plot_data['abundances']],
            textposition='top center',
            textfont=dict(size=14),
            showlegend=False,
            hoverinfo='none'
        ))

        if self.plot_type.value == 'Log Scale':
            default_ylabel = 'Log<sub>10</sub> (Summed Absrobance) ± SEM'
        else:
            default_ylabel = 'Total Absrobance (± SEM)'
        # Update abundance layout
        fig_abundance.update_layout(
            **COMMON_LAYOUT,
            title=create_title(abundance_title),
            xaxis_title=self.xlabel_widget.value or '',
            yaxis_title=self.ylabel_abundance_widget.value or default_ylabel,
            xaxis=AXIS_STYLE,
            yaxis=AXIS_STYLE
        )
        
        # Rest of the plotting code remains the same...
        
        # Configure abundance axes
        fig_abundance.update_xaxes(
            tickangle=45,
            title_font={"size": 18},
            tickfont={"size": 16}
        )
            
        # Set y-axis scale based on plot type
        y_axis_config = {
            'title_font': {"size": 16},
            'tickfont': {"size": 16},
            'gridcolor': "lightgray",
            'showgrid': True
        }
        
        if self.plot_type.value == 'Log Scale':
            y_axis_config.update({
                'type': 'log',
                'exponentformat': 'E',
                'showexponent': 'all',
                'tickformat': ".1e",
                #'dtick': 1,  # Show ticks/gridlines at each order of magnitude (10^1)
            })
        else:  # Linear Scale
            y_axis_config.update({
                'type': 'linear',
                'tickformat': ".1e"
            })
        
        fig_abundance.update_yaxes(**y_axis_config)
        
        # Create count figure
        fig_count = go.Figure()
        
        # Add count bars
        fig_count.add_trace(go.Bar(
            x=groups,
            y=plot_data['counts'],
            name='Peptide Count',
            marker=dict(
                color=self.count_color.value,
                line=dict(color='black', width=1)
            ),
            error_y=dict(
                type='data',
                array=plot_data['count_sems'],
                visible=True,
                thickness=1.5,
                width=4,
                color='#000000'
            ),
            hovertemplate=(
                "Group: %{x}<br>"
                "Unique Peptides: %{y:.0f}<br>"
                "SEM: %{error_y.array:.1f}<br>"
                "<extra></extra>"
            )
        ))
        
        # Add count labels
        fig_count.add_trace(go.Scatter(
            x=groups,
            y=[c + (s * 1.2) for c, s in zip(plot_data['counts'], plot_data['count_sems'])],
            mode='text',
            text=[f"{int(c):,}" for c in plot_data['counts']],
            textposition='top center',
            textfont=dict(size=12),
            showlegend=False,
            hoverinfo='none'
        ))
        
        # Update count layout
        fig_count.update_layout(
            **COMMON_LAYOUT,
            title=create_title(count_title),
            xaxis_title=self.xlabel_widget.value or '',
            yaxis_title=self.ylabel_count_widget.value or 'Unique Peptide Count',
            xaxis=AXIS_STYLE,
            yaxis=AXIS_STYLE
        )
        
        # Configure count axes
        fig_count.update_xaxes(
            tickangle=45,
            title_font={"size": 18},
            tickfont={"size": 16}
        )
        
        fig_count.update_yaxes(
            title_font={"size": 18},
            tickfont={"size": 16},
            tickformat=",d"
        )
        
        return fig_abundance, fig_count

    def _on_plot_button_click(self, b):
        """Handle plot button click"""
        with self.info_output:
            self.info_output.clear_output(wait=True)
            
            if not self.group_select.value:
                display(HTML("<b style='color:red'>Please select at least one group.</b>"))
                return
                
            if self.data_transformer.merged_df is None:
                display(HTML("<b style='color:red'>Please upload data first.</b>"))
                return
                
            if self.data_transformer.group_data is None:
                display(HTML("<b style='color:red'>Please upload group definition first.</b>"))
                return
        
        with self.plot_output:
            self.plot_output.clear_output(wait=True)
            display(HTML("<div style='color:blue'>Generating plots, please wait...</div>"))
            
            try:
                # Process and plot data
                all_data = self._process_total_peptide_data()
                if all_data:
                    # Filter data for selected groups
                    selected_groups = list(self.group_select.value)
                    filtered_data = {k: v for k, v in all_data.items() 
                                   if k in selected_groups}
                    
                    self.current_fig_abundance, self.current_fig_count = self.plot_total_peptides(
                        filtered_data
                    )
                    
                    if self.current_fig_abundance is not None:
                        self.plot_output.clear_output(wait=True)
                        self.current_fig_abundance.show()
                        self.current_fig_count.show()
                        self.export_button.disabled = False
                        self.download_html_button.disabled = False
                    else:
                        display(HTML("<div style='color:red'>Failed to create plots.</div>"))
                        self.export_button.disabled = True
                        self.download_html_button.disabled = True
                else:
                    self.plot_output.clear_output(wait=True)
                    display(HTML("<div style='color:red'>No data available for plotting. Check your filters and group selection.</div>"))
                    self.export_button.disabled = True
                    self.download_html_button.disabled = True
            except Exception as e:
                self.plot_output.clear_output(wait=True)
                display(HTML(f"<div style='color:red'>Error generating plot: {str(e)}</div>"))
                import traceback
                print(traceback.format_exc())
                self.export_button.disabled = True
                self.download_html_button.disabled = True
 
    def export_peptide_data(self, data):
        """
        Export peptide data to Excel with summary and replicate details.
        
        Args:
            data (dict): Dictionary containing peptide analysis results
            
        Returns:
            bytes: Excel file content as bytes
        """
        try:
            # Create summary DataFrame
            summary_data = []
            for group, values in data.items():
                summary_data.append({
                    'Group': group,
                    'Total_Absorbance': values['total_Absorbance'],
                    'Abundance_SEM': values['abundance_sem'],
                    'Unique_Peptides': values['unique_peptides'],
                    'Count_SEM': values['count_sem']
                })
            summary_df = pd.DataFrame(summary_data)
            
            # Create replicate details DataFrame
            replicate_data = []
            for group, values in data.items():
                # Get the replicate information
                replicate_info = values['replicate_data']
                
                # Add entry for each replicate
                for i, replicate_name in enumerate(replicate_info['abundance_columns']):
                    replicate_data.append({
                        'Group': group,
                        'Replicate': replicate_name,
                        'Total_Absorbance': replicate_info['replicate_abundances'][i],
                        'Unique_Peptides': replicate_info['replicate_counts'][i]
                    })
            replicate_df = pd.DataFrame(replicate_data)
            
            # Create Excel file in memory
            output = io.BytesIO()
            with pd.ExcelWriter(output, engine='openpyxl') as writer:
                # Write summary sheet
                summary_df.to_excel(
                    writer, 
                    sheet_name='Summary',
                    index=False
                )
                
                # Write replicate details sheet
                replicate_df.to_excel(
                    writer, 
                    sheet_name='Replicate Details',
                    index=False
                )
                
                # Auto-adjust column widths for both sheets
                for sheet in writer.sheets.values():
                    for column in sheet.columns:
                        max_length = 0
                        column = [cell for cell in column if cell.value is not None]
                        for cell in column:
                            try:
                                if len(str(cell.value)) > max_length:
                                    max_length = len(str(cell.value))
                            except:
                                pass
                        adjusted_width = (max_length + 2)
                        sheet.column_dimensions[column[0].column_letter].width = adjusted_width
            
            # Get the Excel file content
            excel_content = output.getvalue()
            output.close()
            
            return excel_content
            
        except Exception as e:
            print(f"Error exporting data: {str(e)}")
            return None
    
    def _on_export_button_click(self, b):
        """Handle export button click event"""
        try:
            # Get the processed data
            data = self._process_total_peptide_data()
            if data is None:
                with self.export_output:
                    self.export_output.clear_output(wait=True)
                    display(HTML('<div style="color: red; padding: 10px;">No data to export.</div>'))
                return
            
            # Export the data
            excel_content = self.export_peptide_data(data)
            if excel_content is None:
                with self.export_output:
                    self.export_output.clear_output(wait=True)
                    display(HTML('<div style="color: red; padding: 10px;">Error creating export file.</div>'))
                return
            
            # Generate timestamp for filename
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f"Total_Peptide_Analysis_{timestamp}.xlsx"
            
            # Create download link
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'''
                    <div id="export_container_{timestamp}">
                        <a id="export_link_{timestamp}" 
                           href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{base64.b64encode(excel_content).decode()}" 
                           download="{filename}"
                           style="display: none;"></a>
                    </div>
                    <script>
                        document.getElementById('export_link_{timestamp}').click();
                    </script>
                '''))
                
        except Exception as e:
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'<div style="color: red; padding: 10px;">Error during export: {str(e)}</div>'))
    
    def _on_download_html_click(self, b):
        """Handle HTML plot download"""
        if self.current_fig_abundance is None or self.current_fig_count is None:
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML("<div style='color: red; padding: 10px;'>No plots available to download.</div>"))
            return
            
        try:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            
            # Create HTML wrapper with responsive styling
            combined_html = '''
            <html>
            <head>
                <title>Total Peptide Analysis</title>
                <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
                <style>
                    .plot-container {
                        width: 100%;
                        max-width: 1000px;
                        margin: 20px auto;
                    }
                </style>
            </head>
            <body>
            '''
            
            # Add plots with responsive containers
            for plot_type, fig in [('abundance', self.current_fig_abundance), 
                                 ('count', self.current_fig_count)]:
                div_id = f'{plot_type}_chart_{timestamp}'
                combined_html += f'<div class="plot-container" id="{div_id}"></div>\n'
                combined_html += f'<script>Plotly.newPlot("{div_id}", {fig.to_json()});</script>\n'
            
            combined_html += '</body></html>'
            
            # Create download link
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'''
                    <div id="download_container_{timestamp}">
                        <a id="download_link_{timestamp}" 
                           href="data:text/html;charset=utf-8;base64,{base64.b64encode(combined_html.encode()).decode()}" 
                           download="total_peptide_plots_{timestamp}.html"
                           style="display: none;"></a>
                        <script>
                            setTimeout(function() {{
                                document.getElementById('download_link_{timestamp}').click();
                            }}, 100);
                        </script>
                    </div>
                '''))
                
        except Exception as e:
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(f'<div style="color: red; padding: 10px;">Error downloading plots: {str(e)}</div>'))

    def display(self):
        """Display the total peptide analysis interface"""
        display(self.widget_box)

In [4]:

# Define a function to initialize the UI and data properly
def update_abundance_plot(data_transformer):
    """Update the abundance plot based on filtered data"""
    if data_transformer.filtered_df is None:
        return
    
    try:
        # Only attempt to show dropdowns if peptideplotter is defined 
        # and the data is loaded
        if 'peptideplotter' in globals() and data_transformer.merged_df is not None:
            # Set the filter dropdown to show both proteins and functions by default
                peptideplotter.update_display()
    except Exception as e:
        with data_transformer.status_area:
            data_transformer.status_area.clear_output()
            display(HTML(
                f'<div style="color: red; margin-top: 10px;">'
                f'Error updating filters: {str(e)}</div>'
            ))

# Create the instance
data_transformer = DataTransformation()

# Setup data loading UI first
data_transformer.setup_data_loading_ui()

# Create plotter after setting up data transformer
peptideplotter = TotalPeptidePlotter(data_transformer)

# Add the plotting callback - this will initialize the filter display
data_transformer.add_callback(lambda: update_abundance_plot(data_transformer))

# Display the plotter UI
peptideplotter.display()

Output()